In [1]:
#Import dependencies
import pandas as pd
import numpy as np

#File paths for csv's

file_path_schools = "PyCitySchools/Resources/schools_complete.csv"
file_path_students = "PyCitySchools/Resources/students_complete.csv"

#Read csv's
schools_data = pd.read_csv(file_path_schools)
students_data = pd.read_csv(file_path_students)

#Seeing what they have in common
schools_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [2]:
#Seeing what they have in common
students_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## Setting up the dataframe

In [19]:
#Merging the dataframes on school name
school_data_merged = pd.merge(students_data, schools_data, how="left", on=["school_name", "school_name"])
school_data_merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [21]:
#Calculate the total number of schools
tot_schools = school_data_merged['school_name'].nunique()

In [22]:
#Calculate the total number of students
tot_students = '{:,}'.format(school_data_merged['student_name'].count())

In [23]:
#Calculate the total budget
tot_budget = '${:,.2f}'.format(school_data_merged['budget'].unique().sum())

In [24]:
#Calculate the average math score
avg_math_score = school_data_merged['math_score'].mean()

In [25]:
#Calculate the average reading score
avg_reading_score = school_data_merged['reading_score'].mean()

In [26]:
#Calculate the overall passing rate (overall average score)
overall_pass_rate = (avg_math_score + avg_reading_score) / 2

In [27]:
#Calculate the percentage of students with a passing math score (70 or greater)
pass_rate_math = (len(school_data_merged[school_data_merged['math_score'] >= 70]) / school_data_merged.shape[0]) * 100

In [28]:
#Calculate the percentage of students with a passing reading score (70 or greater)
pass_rate_read = (len(school_data_merged[school_data_merged['reading_score'] >= 70]) / school_data_merged.shape[0])*100

In [29]:
#Defining the variable 'district'
district = (tot_schools, tot_students, tot_budget,
            avg_math_score, avg_reading_score, pass_rate_math,
            pass_rate_read, overall_pass_rate)
#Buildning the District Summary dataframe
District_Summary = pd.DataFrame([district], columns= ['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math',
            '% Passing Reading','% Overall Passing Rate'])
#Calling the Dataframe to output results
District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

In [30]:
#Finding 'Per Student Budget'
school_data_merged['Per Student Budget'] = school_data_merged['budget'] / school_data_merged['size']

In [104]:
math_pass = school_data_merged.loc[school_data_merged['math_score'] >= 70, :]
math_pass_count = math_pass.groupby(['school_name']).agg({'math_score': "count"})
mathpasscount = math_pass_count['math_score']

read_pass = school_data_merged.loc[school_data_merged['reading_score'] >= 70, :]
reading_pass_count = read_pass.groupby(['school_name']).agg({'reading_score': "count"})
readingpasscount = reading_pass_count['reading_score']

school_comparison = school_data_merged[['student_name','school_name', 'type', 'size', 'budget','Per Student Budget',
                                          'math_score','reading_score']]

school_comparison = school_comparison.groupby(['school_name']).agg({'type':'first', 
                                                'size':'first', 'budget':'first', 'Per Student Budget':'first',
                                                'math_score': 'mean', 'reading_score':'mean'})
school_comparison.rename(inplace=True, columns={'type':"School Type", 'size':'Total Students',
                                               'budget':'Total School Budget', 
                                                'math_score':'Average Math Score',
                                               'reading_score':'Average Reading Score'})
school_comparison['% Passing Math'] = mathpasscount / school_comparison['Total Students']*100
school_comparison['% Passing Reading'] = readingpasscount / school_comparison['Total Students']*100
school_comparison['% Overall Passing Rate'] = ((mathpasscount / school_comparison['Total Students']) 
                                               + (readingpasscount /school_comparison['Total Students'])) / 2 * 100

school_comparison["Total School Budget"] = school_comparison["Total School Budget"].map("${:,.2f}".format)
school_comparison["Per Student Budget"] = school_comparison["Per Student Budget"].map("${:,.2f}".format)


## Top Performing Schools (By Overall Passing Rate

In [105]:
school_comparison.sort_values(by=['% Overall Passing Rate'], ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Overall Passing Rate)

In [71]:
school_comparison.sort_values(by=['% Overall Passing Rate'], ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [41]:
table = pd.pivot_table(school_data_merged, values='math_score', index=['school_name'],
                    columns=['grade'], aggfunc=np.mean)
table[['9th','10th','11th','12th']]

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade

In [43]:
table = pd.pivot_table(school_data_merged, values='reading_score', index=['school_name'],
                    columns=['grade'], aggfunc=np.mean)
table[['9th','10th','11th','12th']]

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [109]:
school_data_merged['Per Student Budget'].min()

578.0

In [110]:
school_data_merged['Per Student Budget'].max()

655.0

In [111]:
school_data_merged['Per Student Budget'].mean()

629.2935409752362

In [113]:
school_data_merged['Per Student Budget'].median()

638.0

In [114]:
# Creating bins using the min, max, mean and median as a guide
spending_bins = [0, 600, 620, 640, 660]
group_names = ["<$600", "$600-620", "$620-640", "$640-660"]

In [115]:
school_data_merged["Spending Ranges (Per Student)"] = pd.cut(school_data_merged["Per Student Budget"], 
                                                            spending_bins, labels=group_names)
school_data_merged.groupby(['Spending Ranges (Per Student)']).agg({'math_score': 'mean',
                                                                   'reading_score': 'mean'})


,math_score,reading_score
Spending Ranges (Per Student),,
<$600,83.362283,83.912412
$600-620,83.839917,84.044699
$620-640,78.236441,81.559460
$640-660,77.058995,80.958411


## Scores by School Size

In [121]:
school_data_merged['size'].min()

427

In [122]:
school_data_merged['size'].max()

4976

In [123]:
school_data_merged['size'].mean()

3332.9571100331887

In [126]:
school_data_merged['size'].median()

2949.0

In [133]:
#Creating bins based off of the min, max, mean and median
#School size mean is skewed up compared to median so 3000 seems like a good midpoint
size_bins = [0, 2000, 3500, 5000]
group_names = ["Small (<2000)", "Medium (2000-3500)", "Large (3500-5000)"]

In [134]:
math_pass = school_data_merged.loc[school_data_merged['math_score'] >= 70, :]
math_pass_count = math_pass.groupby(['School Size']).agg({'math_score': "count"})
mathpasscount = math_pass_count['math_score']

school_data_merged["School Size"] = pd.cut(school_data_merged["size"], 
                                                            size_bins, labels=group_names)

score_by_size = school_data_merged.groupby(['School Size']).agg({'math_score': 'mean', 'reading_score': 'mean'})
#score_by_size['% Passing Math'] = mathpasscount /score_by_size['School Size']

score_by_size

,math_score,reading_score
School Size,,
Small (<2000),83.436586,83.882857
Medium (2000-3500),78.164034,81.654758
Large (3500-5000),77.070764,80.928365


## Scores by School Type

In [48]:
score_by_type = school_data_merged.groupby(['type']).agg({'math_score': 'mean', 'reading_score': 'mean'})
score_by_type.rename(columns={'math_score':'Average Math Score',
                     'reading_score':'Average Reading Score',})

,Average Math Score,Average Reading Score
type,,
Charter,83.406183,83.902821
District,76.987026,80.962485


In [107]:
school_data_merged['Per Student Budget'].min()

578.0